In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

## YF Financial Scraper
You can also use this template to scrape balance sheet and cashflow statement, just have to enable the url accordingly.

In [20]:
path = "chromedriver.exe"
service = webdriver.chrome.service.Service(path)
service.start()
options = Options()
options.add_argument("--headless") 

ticker = "AAPL" # Choose any ticker you want the financial data of. Here we are using Apple.
url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(ticker,ticker)
#url = "https://finance.yahoo.com/quote/{}/balance-sheet?p={}".format(ticker,ticker)
#url = "https://finance.yahoo.com/quote/{}/cash-flow?p={}".format(ticker,ticker)

driver = webdriver.Chrome(service=service, options = options)
driver.get(url)
driver.implicitly_wait(3)

### clicking dropdown buttons before scraping   
buttons = driver.find_elements(By.XPATH,  '//section[@class="main svelte-e2c64s"]//button')
for button in buttons:
    if button.accessible_name in ["","Follow","Quarterly","Annual","prev","next"]:
        pass
    else:
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable(button)).click()

table = driver.find_element(By.XPATH,  '//div[@class="tableContainer svelte-1pgoo1f"]')
text_blob = table.text.split("\n")
income_st_dir = {}
last_key = None
for count, row in enumerate(text_blob):
    if count == 0:
        heading = row.split()
        column_count = len(heading[1:])
    else:
        if count%(column_count+1) == 1:
            income_st_dir[row] = []
            last_key = row
        else:
            income_st_dir[last_key].append(row)        
df = pd.DataFrame(income_st_dir).T
df.columns = heading[1:]

In [21]:
df

,TTM,9/30/2023,9/30/2022,9/30/2021,9/30/2020
Total Revenue,"381,623,000","383,285,000","394,328,000","365,817,000","274,515,000"
Operating Revenue,"381,623,000","383,285,000","394,328,000","365,817,000","274,515,000"
Cost of Revenue,"207,657,000","214,137,000","223,546,000","212,981,000","169,559,000"
Gross Profit,"173,966,000","169,148,000","170,782,000","152,836,000","104,956,000"
Operating Expense,"55,726,000","54,847,000","51,345,000","43,887,000","38,668,000"
Selling General and Administrative,"25,378,000","24,932,000","25,094,000","21,973,000","19,916,000"
Research & Development,"30,348,000","29,915,000","26,251,000","21,914,000","18,752,000"
Operating Income,"118,240,000","114,301,000","119,437,000","108,949,000","66,288,000"
Net Non Operating Interest Income Expense,--,"-183,000","-106,000","198,000","890,000"
Interest Income Non Operating,--,"3,750,000","2,825,000","2,843,000","3,763,000"


## Key Statistics Scraper

In [6]:
ticker = "AAPL" # Choose any ticker you want the statistics data of. Here we are using Apple.
url = 'https://finance.yahoo.com/quote/{}/key-statistics'.format(ticker)

path = "chromedriver.exe"
service = webdriver.chrome.service.Service(path)
service.start()
options = Options()
options.add_argument("--headless") 

driver = webdriver.Chrome(service=service, options = options)
driver.get(url)
driver.implicitly_wait(0.2)

table = driver.find_element(By.XPATH,  '//div[@class="table-container svelte-104jbnt"]')
text_blob = table.text.split("\n")
financial_metrics = {}
header = text_blob[0].split()

for item in text_blob[1:]:
    parts = item.split()
    key_end = len(parts) - len(header)
    key = ' '.join(parts[:key_end])
    values = parts[key_end:]
    financial_metrics[key] = dict(zip(header, values))

df = pd.DataFrame(financial_metrics).T
df = df.replace({'T': 'E+12', 'B': 'E+09', 'M': 'E+06', 'K': 'E+03'}, regex=True)
df = df.apply(pd.to_numeric, errors='coerce')

In [7]:
df

,Current,3/31/2024,12/31/2023,9/30/2023,6/30/2023,3/31/2023
Market Cap,2.920000e+12,2.650000e+12,2.990000e+12,2.680000e+12,3.050000e+12,2.610000e+12
Enterprise Value,2.950000e+12,2.680000e+12,3.040000e+12,2.720000e+12,3.100000e+12,2.670000e+12
Trailing P/E,2.955000e+01,2.667000e+01,3.141000e+01,2.873000e+01,3.288000e+01,2.800000e+01
Forward P/E,2.899000e+01,2.632000e+01,2.915000e+01,2.577000e+01,2.941000e+01,2.786000e+01
PEG Ratio (5yr expected),2.250000e+00,2.110000e+00,2.310000e+00,2.180000e+00,2.660000e+00,2.790000e+00
Price/Sales,7.780000e+00,6.990000e+00,7.940000e+00,7.100000e+00,8.080000e+00,6.890000e+00
Price/Book,3.926000e+01,3.549000e+01,4.790000e+01,4.417000e+01,4.908000e+01,4.599000e+01
Enterprise Value/Revenue,7.730000e+00,6.960000e+00,7.940000e+00,7.090000e+00,8.060000e+00,6.890000e+00
Enterprise Value/EBITDA,2.219000e+01,2.010000e+01,2.356000e+01,2.146000e+01,2.451000e+01,2.084000e+01
